In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import random
from glob import glob
from tqdm import tqdm
from scipy.io import loadmat

import torch
from torch import nn
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from datetime import datetime
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn import metrics
import json
from sklearn.preprocessing import LabelEncoder

In [2]:
print(os.getcwd())
save_dir = os.getcwd()
os.chdir("..")
os.chdir("..")
main_dir = os.getcwd() 
print(os.listdir(main_dir))
print(save_dir)

/media/mountHDD2/linh/git/HeartResearch/Chapman Dataset/2D_model/Other_model
['Reimplement', 'Data_Explore', 'Diagnostics.xlsx', '.ipynb_checkpoints', '2D_model', '1D_Model', '2D_Loss']
/media/mountHDD2/linh/git/HeartResearch/Chapman Dataset/2D_model/Other_model


In [3]:
# data_dir = "/media/mountHDD2/khoibaocon"
# print(os.listdir(data_dir))
data_dir = "/media/mountHDD3/data_storage/biomedical_data/ecg_data/ECGDataDenoised"
label_file = main_dir + "/Diagnostics.xlsx"

In [4]:
main_df = pd.read_excel(label_file)
main_df.shape
single_main_df = main_df[['FileName', 'Rhythm']]
single_main_df = single_main_df.drop([4419])
label_encoder = LabelEncoder()
single_main_df['Rhythm'] = label_encoder.fit_transform(single_main_df['Rhythm'])
print(single_main_df)

                         FileName  Rhythm
0      MUSE_20180120_121711_19000       5
1      MUSE_20180120_121704_86000       5
2      MUSE_20180113_125357_13000       5
3      MUSE_20180113_134825_04000       7
4      MUSE_20180115_123455_79000       7
...                           ...     ...
10641  MUSE_20181222_204246_47000      10
10642  MUSE_20180115_120332_79000       5
10643  MUSE_20180712_152507_30000       0
10644  MUSE_20180118_181350_17000       5
10645  MUSE_20180116_121646_28000       9

[10645 rows x 2 columns]


In [5]:
# main_df.head()

In [6]:
# main_df["First_label"].value_counts()

In [7]:
# main_df["Second_label"].value_counts()

In [8]:
# main_df["Third_label"].value_counts()

In [9]:

# single_main_df = main_df[main_df["Second_label"].isnull()]
# single_main_df.shape

In [10]:
mat_files = glob(data_dir + "/*")
print(len(mat_files))

10588


In [11]:
single_fns = single_main_df["FileName"].values.tolist()
print(len(single_fns))

10645


In [12]:
single_mat_paths = []
for file in glob(data_dir +"/*"):
    single_mat_paths.append(file)
# single_mat_paths = [data_dir + f"/{x}.csv" for x in single_fns]
# print(os.path.exists(single_mat_paths[0]))

In [13]:
# sample_data = loadmat(single_mat_paths[0])
# sample_data.keys()

In [14]:
# sample_signal_data = sample_data['ECG'][0][0][2]
# sample_signal_data.shape

In [15]:
# sample_sig = torch.randn(1, 12, 32)
# conv_test = nn.Conv1d(12, 12, 3, 1, 1)
# print(conv_test(sample_sig).shape)

In [16]:
class BasicBlock(nn.Module):
    def __init__(self, channel_num):
        super(BasicBlock, self).__init__()
        self.conv_block1 = nn.Sequential(
			nn.Conv1d(channel_num, channel_num, 3, padding=1),
			nn.BatchNorm1d(channel_num),
			nn.ReLU(),
		)
        self.conv_block2 = nn.Sequential(
			nn.Conv1d(channel_num, channel_num, 3, padding=1),
			nn.BatchNorm1d(channel_num),
		)
        self.relu = nn.ReLU()
        torch.nn.init.kaiming_normal_(self.conv_block1[0].weight)
        torch.nn.init.kaiming_normal_(self.conv_block2[0].weight)
        
    def forward(self, x):
        residual = x
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = x + residual
        out = self.relu(x)
        return out

In [17]:
test_basic_block = BasicBlock(2)
sample_sig = torch.randn(1, 2, 32)
print(test_basic_block(sample_sig).shape)

torch.Size([1, 2, 32])


In [18]:
class ResNet(nn.Module):
    def __init__(self, in_channels = 12, type = 18, num_classes = 11):
        super(ResNet, self).__init__()
        self.struc_dict = {
            18: {
                "num_channels" : [64, 128, 256, 512],
                "counts" : [2, 2, 2, 2]
            }
        }
        self.conv1 = nn.Conv1d(in_channels=in_channels, out_channels=64, kernel_size=7, stride=2)
        torch.nn.init.kaiming_normal_(self.conv1.weight)
        self.max1 = nn.MaxPool1d(kernel_size=3, stride=2)
        self.main = nn.Sequential()
        for idx, struc in enumerate(
            zip(
                self.struc_dict[type]["num_channels"], 
                self.struc_dict[type]["counts"]
            )
        ):
            num_channel, cnt = struc
            for i in range(cnt):
                self.main.add_module(f"conv{idx+1}_{i}", BasicBlock(num_channel))
            if idx < len(self.struc_dict[type]["num_channels"]) - 1:
                self.main.add_module(f"ext_{idx}", nn.Conv1d(num_channel, self.struc_dict[type]["num_channels"][idx+1], 3, 1))
                self.main.add_module(f"extbn_{idx}", nn.BatchNorm1d(self.struc_dict[type]["num_channels"][idx+1]))
                                     
        self.avg = torch.nn.AdaptiveAvgPool1d((1))
        self.lin = nn.Linear(self.struc_dict[type]["num_channels"][-1], num_classes)
        torch.nn.init.kaiming_normal_(self.lin.weight)
    def forward(self, x):
        x = self.conv1(x)
        x = self.max1(x)
        x = self.main(x)
        x = self.avg(x)
        x = x.reshape(x.shape[0], -1)
        x = self.lin(x)
        return x

In [19]:
model = ResNet()
sample_sig = torch.randn(1, 12, 3000)
model(sample_sig).shape

torch.Size([1, 11])

In [20]:
class ECG(Dataset):
    def __init__(self, data_paths, label_df):
        self.data_paths = data_paths
        random.shuffle(self.data_paths)
        self.label_df = label_df

    def __getitem__(self, idx):
        data_path = self.data_paths[idx]        
        data = pd.read_csv(data_path, header = None)
        data = data.values.T
        clip_data = data[:, 500:3000]

        filename = data_path.split("/")[-1].split(".")[0]
        label = self.label_df[self.label_df["FileName"] == filename]["Rhythm"].values.item()

        torch_data = torch.from_numpy(clip_data)

        return torch_data.float(), label

    def __len__(self):
        return len(self.data_paths)

In [21]:
# check_ds = ECG(data_paths=single_mat_paths, label_df=single_main_df)
# sample, lbl = check_ds[0]
# print(sample.shape, lbl)

In [22]:
# model(sample.unsqueeze(dim=0)).shape

In [23]:
data_dict = {
    idx : [] for idx in range(11)
}

for data_path in single_mat_paths:
    filename = data_path.split("/")[-1].split(".")[0]
    _cls = single_main_df[single_main_df["FileName"] == filename]["Rhythm"].values.item()

    data_dict[_cls].append(data_path)

for key in data_dict:
    print(f"{key}->{len(data_dict[key])}")

0->438
1->1780
2->121
3->16
4->8
5->397
6->7
7->3888
8->1825
9->1564
10->544


In [24]:
train_data_dict = {
    _cls : data_dict[_cls][:int(0.9*len(data_dict[_cls]))] for _cls in data_dict
}

valid_data_dict = {
    _cls : data_dict[_cls][int(0.9*len(data_dict[_cls])):] for _cls in data_dict
}

for key in train_data_dict:
    print(f"{key}->{len(train_data_dict[key])}--{len(valid_data_dict[key])}")

0->394--44
1->1602--178
2->108--13
3->14--2
4->7--1
5->357--40
6->6--1
7->3499--389
8->1642--183
9->1407--157
10->489--55


In [25]:
train_data_paths = []
for key in train_data_dict:
    train_data_paths.extend(train_data_dict[key])
valid_data_paths = []
for key in valid_data_dict:
    valid_data_paths.extend(valid_data_dict[key])
print(len(train_data_paths))
print(len(valid_data_paths))

9525
1063


In [26]:
train_ds = ECG(train_data_paths, single_main_df)
valid_ds = ECG(valid_data_paths, single_main_df)

print(len(train_ds))
print(len(valid_ds))

9525
1063


In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu", index = 1)
# batch_size = 32
batch_size = 64

traindl = DataLoader(
    train_ds,
    batch_size=batch_size, 
    shuffle=True, 
    pin_memory=True, 
    num_workers=os.cpu_count()//2
    # num_workers = 0
)

validdl = DataLoader(
    valid_ds,
    batch_size=1, 
    shuffle=True, 
    pin_memory=True, 
    num_workers=os.cpu_count()//2
    # num_workers = 0
)

print(len(traindl))
print(len(validdl))

149
1063


In [28]:
class FocalClassifierV0(nn.Module):
    def __init__(self, gamma=0.3): #Change gamma value here in order to acquire other results
        super().__init__()
        
        self.gamma = gamma
        self.act = nn.LogSoftmax(dim=1)

    
    def forward(self, pred, target):

        logits = self.act(pred)

        B, C = tuple(logits.size())

        entropy = torch.pow(1 - logits, self.gamma) * logits * F.one_hot(target, num_classes=C).float()

        return (-1 / B) * torch.sum(entropy)

focalloss_fn = FocalClassifierV0()

In [29]:
epoch = 150
lr = 0.001
model.to(device)
optimizer = Adam(model.parameters(), lr=lr)
scheduler = CosineAnnealingLR(optimizer=optimizer, T_max=epoch)
loss_fn = nn.CrossEntropyLoss()
best_acc = 0
best_ep = 0
checkpoint_folder = "run_ResNet_0.05_gamma0.3_lr0001_10"

In [30]:
def loss_plot(train_losses, val_losses, n_epochs, check_folder):
    now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
    run_dir = save_dir + f"/{check_folder}"
    if not os.path.exists(run_dir):
        os.mkdir(run_dir)
    save_loss_dir = run_dir + "/save_losses"
    if not os.path.exists(save_loss_dir):
        os.mkdir(save_loss_dir)
    save_fig_losses = os.path.join(save_loss_dir, f"plot_losses_epoch{n_epochs}_{now}.png")  

    plt.figure()
    plt.plot(range(epoch), np.array(train_losses), label='Train Loss')
    plt.plot(range(epoch), np.array(val_losses), label='Test Loss')
    plt.xlabel('Epoch')
    # plt.xticks()
    plt.ylabel('Loss')
    plt.title('Training and Test Loss')
    plt.legend()
    plt.savefig(save_fig_losses)
    

def acc_plot(train_cls_acc, val_cls_acc, n_epochs, check_folder):
    now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
    run_dir = save_dir + f"/{check_folder}"
    if not os.path.exists(run_dir):
        os.mkdir(run_dir)
#         save_dir = run_dir + f"/{now}"
#         if not os.path.exists(save_dir):
#             os.mkdir(save_dir)
    save_acc_dir = run_dir + "/save_acc"
    if not os.path.exists(save_acc_dir):
        os.mkdir(save_acc_dir)
    save_fig_acc = os.path.join(save_acc_dir, 'plot_acc_epoch{}_{}.png'.format(n_epochs, now))  

    plt.figure()
    plt.plot(range(epoch), np.array(train_cls_acc), label='Train Accuracy')
    plt.plot(range(epoch), np.array(val_cls_acc), label='Test Accuracy')
    plt.xlabel('Epoch')
    # plt.xticks
    plt.ylabel('Accuracy')
    plt.title('Training and Test Accuracy')
    plt.legend()
    plt.savefig(save_fig_acc)

def checkpoint(valid_class_acc, 
               val_total_loss,
               old_valid_class_acc,
               old_valid_loss,
               epoch, 
               model,
               optimizer,
               check_folder
#                    logs
              ):

    if valid_class_acc >= old_valid_class_acc and val_total_loss <= old_valid_loss:
        old_valid_class_acc = valid_class_acc
        old_valid_loss = val_total_loss
        save_dict = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': val_total_loss,
            'test_acc': valid_class_acc
        }

     # Saving best model
        now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
        run_dir = save_dir + f"/{check_folder}"
        if not os.path.exists(run_dir):
            os.mkdir(run_dir)
    #         save_dir = run_dir + f"/{now}"
    #         if not os.path.exists(save_dir):
    #             os.mkdir(save_dir)
        save_best_model_dir = run_dir + "/save_best_model"
        if not os.path.exists(save_best_model_dir):
            os.mkdir(save_best_model_dir)
        save_best_model_path = save_best_model_dir + f"/{save_dict['loss']:>7f}_{save_dict['test_acc']:>7f}_{now}.pt"
        torch.save(save_dict, save_best_model_path)
        
def classification_report_csv(report, auc, check_folder):
    now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
    run_dir = save_dir + f"/{check_folder}"
    if not os.path.exists(run_dir):
        os.mkdir(run_dir)
    save_report_dir = run_dir + "/save_classification_report"
    if not os.path.exists(save_report_dir):
        os.mkdir(save_report_dir)
        
    report_data = report['macro avg']
    del report_data['support']
    report_data.update({'auc': auc})
#     print(type(report_data))
#     print(report_data)

#     dataframe = pd.DataFrame.from_dict(report_data, orient='index')
#     save_report_file = save_report_dir + f"/classification_report_{now}.csv"
#     dataframe.to_csv(save_report_file, index = False)
    with open(save_report_dir + f"/cls_report_{now}.json", "w") as outfile: 
        json.dump(report_data, outfile)
    
def acc_loss_json(log_dict, check_folder):
    now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
    run_dir = save_dir + f"/{check_folder}"
    if not os.path.exists(run_dir):
        os.mkdir(run_dir)
    save_json_dir = run_dir + "/save_acc_loss_json"
    if not os.path.exists(save_json_dir):
        os.mkdir(save_json_dir) 
    save_acc_loss_file = save_json_dir + f"/acc_loss_{now}.json"
    with open(save_acc_loss_file, "w") as outfile: 
        json.dump(log_dict, outfile)

In [31]:
log_dict = {
    "train": {
        "acc": [],
        "loss": []
    },
    "valid": {
        "acc": [],
        "loss": []
    }
}
train_losses = []
train_acc_plot = []
val_losses = []
val_acc_plot = []
old_valid_class_acc = 0
old_valid_loss = 1e23

In [32]:
for e in range(epoch):
    model.train()
    print(f"Epoch: {e}")
    batch_cnt = 0
    total_loss = 0
    correct = 0
    for batch, (train_sig, train_label) in tqdm(enumerate(traindl)):
        batch_cnt = batch
        train_sig = train_sig.to(device)
        train_label = train_label.to(device)
        
        pred = model(train_sig)
        loss = focalloss_fn(pred, train_label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        scheduler.step()
        
        total_loss += loss.item()
        correct += (pred.argmax(1) == train_label).type(torch.float).sum().item()
    
    total_loss /= batch_cnt
    correct /= len(traindl.dataset)
    
    print(f"train loss: {total_loss} - train acc: {100*correct}")
    
    batch_cnt = 0
    val_total_loss = 0
    val_correct = 0
    model.eval()
    y_true_list = [] 
    pred_list = []
    with torch.no_grad():
        for batch, (valid_sig, valid_label) in tqdm(enumerate(validdl)):
            batch_cnt = batch
            valid_sig = valid_sig.to(device)
            valid_label = valid_label.to(device)
            
            pred = model(valid_sig)
            pred_pos = pred.argmax(1)
            y_true_list.append(valid_label)
            pred_list.append(pred_pos)
            loss = loss_fn(pred, valid_label)
            
            val_total_loss += loss.item()
            val_correct += (pred.argmax(1) == valid_label).type(torch.float).sum().item()
    
        val_total_loss /= batch_cnt
        val_correct /= len(validdl.dataset)
        log_dict["valid"]["loss"].append(val_total_loss)
        log_dict["valid"]["acc"].append(val_correct)
        
        if val_correct > best_acc:
            best_acc = val_correct
            best_ep = e
        
        print(f"valid loss: {val_total_loss} - valid acc: {100*val_correct}")
        if val_correct >= old_valid_class_acc:
            old_valid_class_acc = val_correct
            old_valid_loss = val_total_loss
            save_dict = {
                'epoch': e,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': val_total_loss,
                'test_acc': val_correct
            }

         # Saving best model
            now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
            run_dir = save_dir + f"/{checkpoint_folder}"
            if not os.path.exists(run_dir):
                os.mkdir(run_dir)
            save_best_model_dir = run_dir + "/save_best_model"
            if not os.path.exists(save_best_model_dir):
                os.mkdir(save_best_model_dir)
            save_best_model_path = save_best_model_dir + f"/{save_dict['loss']:>7f}_{save_dict['test_acc']:>7f}_{now}.pt"
            torch.save(save_dict, save_best_model_path)        
print(f"Best acuracy: {best_acc} at epoch {best_ep}")
y_true = torch.cat(y_true_list).cpu().numpy()
pred = torch.cat(pred_list).cpu().numpy()

# Save loss and acc to json file
acc_loss_json(log_dict, check_folder = checkpoint_folder)

# AUC
fpr, tpr, thresholds = metrics.roc_curve(y_true, pred, pos_label = 0)
# print(fpr)
# print(f"tpr: {tpr}")
auc1 = metrics.auc(fpr, tpr)

reports = classification_report(y_true, pred, output_dict=True) 
# print(reports)
classification_report_csv(report = reports, auc = auc1, check_folder = checkpoint_folder)

Epoch: 0


0it [00:00, ?it/s]

1it [00:02,  2.37s/it]

3it [00:02,  1.51it/s]

5it [00:02,  2.83it/s]

...

1019it [00:04, 211.96it/s]

1043it [00:04, 219.88it/s]

1063it [00:05, 206.39it/s]

valid loss: 0.7388772518649394 - valid acc: 85.41862652869237
Best acuracy: 0.8767638758231421 at epoch 26



/media/mountHDD2/linh/git/.env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/mountHDD2/linh/git/.env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/mountHDD2/linh/git/.env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
